In [1]:
from cvrpy.pso import PSO

pso = PSO(
    lambda vec: 1/sum(vec),
    num_iterations=100,
    num_particles=100,
    num_dimensions=20
)

solution = pso.optimize()

print(type(solution))
print(solution.position)


<class 'cvrpy.pso.Particle'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [17]:
from loggibud.v1.types import CVRPInstance
from cvrpy.particle.decoder import ParticleDecoder
from cvrpy.particle.factory import ParticleFactory

instance = CVRPInstance.from_file('data/cvrp_castanhal.json')
print(f'Número de clientes: {len(instance.deliveries)}')
print(f'Capacidade máxima por veículo: {instance.vehicle_capacity}')

# Cria uma partícula e decodifica gerando um CVRPSolution
particle = ParticleFactory.create_for(instance, sum, num_vehicles=5)
solution = ParticleDecoder.decode(particle, instance)

for vehicle in solution.vehicles:
    sizes = [d.size for d in vehicle.deliveries]
    print(f'Total = {sum(sizes)}', sizes, f'{len(sizes)} clientes')


Número de clientes: 35
Capacidade máxima por veículo: 180
Total = 0 [] 0 clientes
Total = 0 [] 0 clientes
Total = 35 [2, 8, 2, 6, 8, 9] 6 clientes
Total = 40 [1, 10, 9, 4, 4, 1, 1, 2, 1, 7] 10 clientes
Total = 119 [3, 6, 4, 7, 5, 3, 10, 7, 10, 10, 9, 7, 4, 2, 8, 7, 4, 4, 9] 19 clientes


## Função Objetivo

Deve calcular a distância percorrida em cada rota, para isso será usado o OSRM
como backend fornecedor dos cálculos de distâncias percorridas entre um ponto
e outro no mapa.

Dado um objeto de CVRPSolution que possui as entregas distribuídas entre os
veículos, pra cada veículo, a função deve calcular a distância entre o depósito
e a primeira entrega, depois entre a primeira e a segunda entrega e assim por
diante até voltar da última entrega para o depósito.

In [48]:
import requests
import json
from loggibud.v1.types import CVRPSolutionVehicle, Point, CVRPInstance

instance = CVRPInstance.from_file('data/cvrp_castanhal.json')
base_url = "http://localhost:5000"

def request_distance_between(a: Point, b: Point) -> float:
    url = f'{base_url}/route/v1/drive/{a.lng},{a.lat};{b.lng},{b.lat}'

    response = requests.get(url, params={
            'alternatives': 'false',
            'steps': 'false',
            'overview': 'false'
        })

    if 200 <= response.status_code < 300:
        data = json.loads(response.content)
        # print(f'{a} -> {b}, distância = {data["routes"][0]["distance"]}')
        return data['routes'][0]['distance']
    else:
        print(f'Erro na requisição GET {response.url}')
        return 0.

# request_distance_between(instance.origin, instance.deliveries[1].point)

In [52]:
from cvrpy.particle.decoder import ParticleDecoder
from cvrpy.particle.factory import ParticleFactory

# Calcula a distância percorrida por um mesmo veículo partindo da origem e
# passando por todos os pontos de entrega, e por último voltando para a origem
def vehicle_distance_traveled(vehicle: CVRPSolutionVehicle) -> float:
    distance = 0.

    if len(vehicle.deliveries) > 0:
        distance += request_distance_between(vehicle.origin, vehicle.deliveries[0].point)
        for i, delivery in enumerate(vehicle.deliveries):
            next_i = i+1
            if next_i < len(vehicle.deliveries):
                next_delivery = vehicle.deliveries[next_i]
                distance += request_distance_between(delivery.point, next_delivery.point)
            pass

        distance += request_distance_between(vehicle.deliveries[-1].point, vehicle.origin)

    return distance


# Cria uma partícula e decodifica gerando um CVRPSolution
particle = ParticleFactory.create_for(instance, sum, num_vehicles=5)
solution = ParticleDecoder.decode(particle, instance)

for i, vehicle in enumerate(solution.vehicles):
    print(f'Veículo {i+1}')
    print(f'\tNúmero de entregas: {len(vehicle.deliveries)}')
    print(f'\tDistância total: {vehicle_distance_traveled(vehicle)}')
    print(f'\tPeso: {vehicle.occupation}')

vehicle_distance_traveled(solution.vehicles[0])

Veículo 1
	Número de entregas: 29
	Distância total: 64194.7
	Peso: 159
Veículo 2
	Número de entregas: 3
	Distância total: 109009.4
	Peso: 19
Veículo 3
	Número de entregas: 3
	Distância total: 66367.3
	Peso: 16
Veículo 4
	Número de entregas: 0
	Distância total: 0.0
	Peso: 0
Veículo 5
	Número de entregas: 0
	Distância total: 0.0
	Peso: 0


64194.7